Чтение данных

In [1]:
import os
import shutil
import sys
import glob
import pickle
import numpy as np

from pathlib import Path

In [2]:
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, DenseFeat,get_feature_names

In [3]:
DATA_PATH = Path('..') / '..' / 'data' / 'criteo'

TRAIN_PATH = str(DATA_PATH / 'train.csv')

In [4]:
import pandas as pd

In [5]:
train = pd.read_csv(TRAIN_PATH)

In [6]:
train.head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,id
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0.0,4.0,...,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN,12
1,1,0.0,1,20.0,16.0,1548.0,93.0,42.0,32.0,912.0,...,1f868fdd,21ddcdc9,a458ea53,7eee76d1,NaN,32c7478e,9af06ad9,9d93af03,cdfe5ab7,26
2,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23.0,24.0,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,NaN,32c7478e,94bde4f2,010f6491,09b76f8d,39
3,1,88.0,319,NaN,4.0,5.0,4.0,89.0,40.0,88.0,...,bbf70d82,NaN,NaN,16e2e3b3,NaN,32c7478e,d859b4dd,NaN,NaN,41
4,0,0.0,53,NaN,10.0,6550.0,98.0,34.0,11.0,349.0,...,fa0643ee,21ddcdc9,b1252a9d,0094bc78,NaN,32c7478e,29ece3ed,001f3601,402185f3,85


In [7]:
train = train.rename(columns=lambda x: x.replace('_', '').upper())

In [8]:
train.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C31,C32,C33,C34,C35,C36,C37,C38,C39,ID
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0.0,4.0,...,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN,12
1,1,0.0,1,20.0,16.0,1548.0,93.0,42.0,32.0,912.0,...,1f868fdd,21ddcdc9,a458ea53,7eee76d1,NaN,32c7478e,9af06ad9,9d93af03,cdfe5ab7,26
2,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23.0,24.0,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,NaN,32c7478e,94bde4f2,010f6491,09b76f8d,39
3,1,88.0,319,NaN,4.0,5.0,4.0,89.0,40.0,88.0,...,bbf70d82,NaN,NaN,16e2e3b3,NaN,32c7478e,d859b4dd,NaN,NaN,41
4,0,0.0,53,NaN,10.0,6550.0,98.0,34.0,11.0,349.0,...,fa0643ee,21ddcdc9,b1252a9d,0094bc78,NaN,32c7478e,29ece3ed,001f3601,402185f3,85


In [9]:
num_columns = ['C{}'.format(i) for i in range(1, 14)]
cat_columns = ['C{}'.format(i) for i in range(14, 40)]
target = ['C0']
len(num_columns), len(cat_columns)

(13, 26)

In [14]:
X, y = train.drop(columns=['C0', 'ID']), train['C0']

In [15]:
from sklearn.preprocessing import MinMaxScaler

In [16]:
mms = MinMaxScaler(feature_range=(0, 1))

In [17]:
X_encoded = X

In [18]:
X_encoded[num_columns] = mms.fit_transform(X_encoded[num_columns])

In [25]:
X_encoded[num_columns] = X_encoded[num_columns].fillna(0)
X_encoded[cat_columns] = X_encoded[cat_columns].fillna('-1')

In [ ]:
X_encoded[num_columns] = X_encoded[num_columns].astype(np.float32)
X_encoded[cat_columns] = X_encoded[cat_columns].astype('string')

In [26]:
X_encoded.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39
0,0.000000,0.000091,0.000000,0.000000,0.000558,0.000000,0.001717,0.000000,0.000207,0.000000,...,e5ba7672,e5f8f18f,-1,-1,f3ddd519,-1,32c7478e,b34f3128,-1,-1
1,0.000000,0.000181,0.000305,0.018244,0.000590,0.000798,0.004242,0.006335,0.047188,0.000000,...,27c07bd6,1f868fdd,21ddcdc9,a458ea53,7eee76d1,-1,32c7478e,9af06ad9,9d93af03,cdfe5ab7
2,0.001385,0.000136,0.000229,0.022805,0.000044,0.000206,0.000808,0.004554,0.001242,0.222222,...,8efede7f,1304f63b,21ddcdc9,b1252a9d,07b2853e,-1,32c7478e,94bde4f2,010f6491,09b76f8d
3,0.015238,0.014591,0.000000,0.004561,0.000002,0.000034,0.008989,0.007919,0.004553,0.333333,...,8efede7f,bbf70d82,-1,-1,16e2e3b3,-1,32c7478e,d859b4dd,-1,-1
4,0.000000,0.002537,0.000000,0.011403,0.002497,0.000841,0.003434,0.002178,0.018058,0.000000,...,e5ba7672,fa0643ee,21ddcdc9,b1252a9d,0094bc78,-1,32c7478e,29ece3ed,001f3601,402185f3


In [27]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=1000,embedding_dim=4, use_hash=True, dtype='string')  # since the input is string
                              for feat in cat_columns] + [DenseFeat(feat, 1, )
                          for feat in num_columns]

In [28]:
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [30]:
train_model_input = {name:X_encoded[name] for name in feature_names}

In [31]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')

In [32]:
model.compile("adam", "binary_crossentropy",
                  metrics=['binary_crossentropy'])

In [33]:
history = model.fit(train_model_input, y.values,
                        batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Train on 2931944 samples, validate on 732987 samples
Epoch 1/10


C:\Anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2931944/2931944 - 158s - loss: 0.4820 - binary_crossentropy: 0.4794 - val_loss: 0.4802 - val_binary_crossentropy: 0.4762
Epoch 2/10
2931944/2931944 - 154s - loss: 0.4729 - binary_crossentropy: 0.4681 - val_loss: 0.4780 - val_binary_crossentropy: 0.4729
Epoch 3/10
2931944/2931944 - 156s - loss: 0.4699 - binary_crossentropy: 0.4642 - val_loss: 0.4775 - val_binary_crossentropy: 0.4718
Epoch 4/10
2931944/2931944 - 157s - loss: 0.4681 - binary_crossentropy: 0.4620 - val_loss: 0.4776 - val_binary_crossentropy: 0.4716
Epoch 5/10
2931944/2931944 - 156s - loss: 0.4669 - binary_crossentropy: 0.4605 - val_loss: 0.4775 - val_binary_crossentropy: 0.4712
Epoch 6/10
2931944/2931944 - 156s - loss: 0.4660 - binary_crossentropy: 0.4593 - val_loss: 0.4773 - val_binary_crossentropy: 0.4708
Epoch 7/10
2931944/2931944 - 157s - loss: 0.4652 - binary_crossentropy: 0.4584 - val_loss: 0.4773 - val_binary_crossentropy: 0.4707
Epoch 8/10
2931944/2931944 - 157s - loss: 0.4645 - binary_crossentropy: 0.4575 - val_lo

In [34]:
TEST_PATH = str(DATA_PATH / 'test.csv')

In [51]:
test = pd.read_csv(TEST_PATH)

In [52]:
X_test = test.drop(columns=['id'])

In [53]:
X_test = X_test.rename(columns=lambda x: x.replace('_', '').upper())

In [38]:
X_test[num_columns] = mms.fit_transform(X_test[num_columns])

In [39]:
X_test[num_columns] = X_test[num_columns].fillna(0)
X_test[cat_columns] = X_test[cat_columns].fillna('-1')

In [40]:
X_test[num_columns] = X_test[num_columns].astype(np.float32)
X_test[cat_columns] = X_test[cat_columns].astype('string')

In [41]:
X_test.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39
0,0.0,0.001315,0.000044,0.005682,0.001843,0.000109,0.000657,0.000971,0.000851,0.0,...,e5ba7672,43de85d3,-1,-1,b64021bd,-1,32c7478e,f1a27f66,-1,-1
1,0.0,0.000188,0.000022,0.000000,0.002291,0.000000,0.000000,0.000388,0.000568,0.0,...,e5ba7672,e7e991cb,efa3470f,a458ea53,6ef75f1d,78e2e389,32c7478e,f53ea242,cb079c2d,e9b68fcc
2,0.0,0.027988,0.000044,0.002841,0.003456,0.000472,0.000110,0.000388,0.001476,0.0,...,d4bb7bd8,e5f8f18f,-1,-1,f3ddd519,ad3062eb,32c7478e,b34f3128,-1,-1
3,0.0,0.010895,0.000156,0.001420,0.000809,0.002594,0.002629,0.005437,0.024408,0.0,...,e5ba7672,eef7297e,-1,-1,8ae05402,ad3062eb,423fab69,8d4a9014,-1,-1
4,0.0,0.000814,0.000089,0.005682,0.000006,0.000000,0.000000,0.000777,0.000341,0.0,...,e5ba7672,7181ccc8,-1,-1,2265e99d,-1,32c7478e,5dc43b96,-1,-1


In [42]:
test_model_input = {name:X_test[name] for name in feature_names}

In [43]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [54]:
test['proba'] = pred_ans

In [64]:
SUBMISSION_PATH = 'deepfm_full_dataset.csv'

In [65]:
test[["id", "proba"]].to_csv(SUBMISSION_PATH, index=False)

In [70]:
test[["id", "proba"]].head()

,id,proba
0,566935904713,0.410206
1,566935904715,0.367769
2,566935904727,0.405519
3,566935904737,0.510916
4,566935904741,0.380276


In [72]:
! kaggle competitions submissions mlbd-20-ctr-prediction-1 -v

fileName,date,description,status,publicScore,privateScore

deepfm_full_dataset.csv,2020-03-31 23:21:22,"full train, hash, deepfm",complete,0.77674,None

full_dataset_mte_xgb.csv,2020-03-31 21:58:04,"full train, mte, xgb, tuned",complete,0.77880,None

hyperopt_xgb.csv,2020-03-12 06:31:23,"still 0.5 of train, xgb, but tuned with hyperopt",complete,0.73301,None

pure_xgb.csv,2020-03-12 06:30:03,"still 0.5 of train, xgb, without tuning",complete,0.71998,None

hyperopt_logreg.csv,2020-03-12 06:25:26,"still 0.5 of train, logreg, but tuned with hyperopt",complete,0.69511,None

hyperopt_logreg.csv,2020-03-12 06:20:56,error_sub,error,None,None

submission.csv,2020-02-27 01:26:19,"Random half of train, moreover 0.8 of it, log reg with 50 iterations. Local AUC: ~70",complete,0.69420,None

